<a href="https://colab.research.google.com/github/raghavmayank/ML/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1: Data Preprocessing.
- 10 Marks
A csv file has been provided to you at this link(https://drive.google.com/file/d/1r3VZs0B54gNvaar9QmTbaHsV75p7b23l/view?usp=sharing). The given dataset is related to cars and contains
26 columns. In the given dataset, “Price” is the target variable (i.e., the output). The marks
distribution according to the tasks are as follows:
1. Assign a type to each of the following features (a) Model, (b) Type, (c) Max. Price and (d)
Airbags from the following: ordinal/nominal/ratio/interval scale.